### import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### import the data

In [ ]:
raw_data = pd.read_csv('../input/boston-housing-dataset/HousingData.csv')
raw_data.head()

In [ ]:
raw_data.describe()

The data has missing values for CRIM, ZN, INDUS, CHAS, AGE, LSTAT

In [ ]:
sns.pairplot(raw_data)

the data is also not normally ditributed
data lacks linear relation among many parameters

### Treating the missing data

We replace mean with missing values of AGE, LSTAT
Since mean is far away from median we will replace missing values with median for CRIM, ZN, INDUS
Since CHAS shows no relation to any data we can drop it

In [ ]:
data1 = raw_data

In [ ]:
data1['AGE']  = data1['AGE'].replace(np.NaN , data1['AGE'].mean())

In [ ]:
data1['LSTAT']  = data1['LSTAT'].replace(np.NaN , data1['LSTAT'].mean())

In [ ]:
data1['CRIM']  = data1['CRIM'].replace(np.NaN , data1['CRIM'].median())

In [ ]:
data1['ZN']  = data1['ZN'].replace(np.NaN , data1['ZN'].median())

In [ ]:
data1['INDUS']  = data1['INDUS'].replace(np.NaN , data1['INDUS'].median())

In [ ]:
data1 = data1.drop(['CHAS'] , axis=1)

In [ ]:
data1.describe()

In [ ]:
data1.reset_index(drop=True)

### Checking for OLS assumptions

In [ ]:
trial_crim = np.log(data1['CRIM'])

In [ ]:
sns.distplot(trial_crim)

zn cannot be converted to log scale as a lot of values are 0

In [ ]:
trial_age = np.log(data1['AGE'])
sns.distplot(trial_age)

age is stil not normal

In [ ]:
trial_dis = np.log(data1['DIS'])
sns.distplot(trial_dis)

b is already squared deviation  hence no changes suggested

In [ ]:
trial_nox = np.log(data1['NOX'])
sns.distplot(trial_nox)

In [ ]:
trial_lstat = np.log(data1['LSTAT'])
sns.distplot(trial_lstat)

dding columns where conversion has made a difference

In [ ]:
data1.describe()

In [ ]:
data1['logCRIM'] = trial_crim

In [ ]:
data1['logDIS'] = trial_dis

In [ ]:
data1['logNOX'] = trial_nox

In [ ]:
data1['logLSTAT'] = trial_lstat

In [ ]:
data1.describe()

In [ ]:
q =data1['AGE'].quantile(0.01)
print(q)

In [ ]:
daa1 = data1[data1['AGE']>q]

In [ ]:
sns.distplot(daa1['AGE'])

In [ ]:
daa1.describe()

In [ ]:
daa1.reset_index(drop=True)

In [ ]:
daa1['logAGE'] = np.log(daa1['AGE'])

In [ ]:
sns.pairplot(daa1)

In [ ]:
sns.heatmap(daa1.corr(), cmap = 'vlag')

### Checking for multicollinearity

In [ ]:
data2 = daa1.drop(['logAGE'] , axis=1)

In [ ]:
data2.reset_index(drop= True)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
variables = data2[['TAX','logCRIM','logDIS' ,'logNOX' ]]

In [ ]:
vif = pd.DataFrame()

In [ ]:
vif['features'] = variables.columns.values

In [ ]:
vif['values'] = [variance_inflation_factor(variables.values , i) for i in range(variables.shape[1])]

In [ ]:
vif

In [ ]:
variables1 = data2[['TAX','logCRIM','DIS' ,'NOX' ]]
vif1 = pd.DataFrame()
vif1['features'] = variables1.columns.values
vif1['values'] = [variance_inflation_factor(variables1.values , i) for i in range(variables1.shape[1])]

In [ ]:
vif1

In [ ]:
variables2 = data2[['TAX','logCRIM','DIS' ,'logLSTAT' ]]
vif2 = pd.DataFrame()
vif2['features'] = variables2.columns.values
vif2['values'] = [variance_inflation_factor(variables2.values , i) for i in range(variables2.shape[1])]

In [ ]:
vif2

based on the vif results we drop a few features

In [ ]:
data3= data2.drop(['NOX' , 'logNOX'  ] , axis=1)

In [ ]:
data3.reset_index(drop=True)

In [ ]:
variables3 = data3[['logCRIM','logDIS' ,'logLSTAT' ]]
vif3 = pd.DataFrame()
vif3['features'] = variables3.columns.values
vif3['values'] = [variance_inflation_factor(variables3.values , i) for i in range(variables3.shape[1])]

In [ ]:
vif3

In [ ]:
data4 = data3.drop(['TAX'] , axis=1)

In [ ]:
data4.reset_index(drop=True)

In [ ]:
data4.columns.values

In [ ]:
data5 = data4[['logCRIM','ZN','INDUS','RM','AGE','logDIS','RAD','PTRATIO','B','logLSTAT','MEDV']]

In [ ]:
data5.head()

# Model 1

### Standardisation of the data

In [ ]:
data_preprocessed = data5

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler1 = StandardScaler()

In [ ]:
scaler1.fit(data_preprocessed)

In [ ]:
data_scaled1 = scaler1.transform(data_preprocessed)

In [ ]:
scaler1.transform(data_preprocessed)

In [ ]:
x1 = data_scaled1[:,0:-1]
y1 = data_scaled1[:,-1]

In [ ]:
x1

In [ ]:
x1.shape

In [ ]:
y1

In [ ]:
y1.shape

### Creating train test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x1_train,x1_test, y1_train, y1_test = train_test_split(x1,y1,test_size=0.2, shuffle=True, random_state = 42)

In [ ]:
x1_train

### Creating the linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model1 = LinearRegression()

In [ ]:
model1.fit(x1_train, y1_train)

In [ ]:
model1.score(x1_train,y1_train)

In [ ]:
r2 = model1.score(x1_train,y1_train)
r2

In [ ]:
n = x1_train.shape[0]
n

In [ ]:
p = x1_train.shape[1]
p

In [ ]:
adj_r2 = 1-((1-r2)*(n-1)/(n-p-1))

In [ ]:
adj_r2

In [ ]:
test_score = model1.score(x1_test,y1_test)

In [ ]:
test_score

In [ ]:
summary = pd.DataFrame()

In [ ]:
features =['logCRIM','ZN','INDUS','RM','AGE','logDIS','RAD','PTRATIO','B','logLSTAT']


In [ ]:
summary['features'] = features
summary['Values']= model1.coef_

In [ ]:
summary

In [ ]:
model1.intercept_

# Model 2

here we will not standardise yonly x

### Standardisation of data

In [ ]:
unscaled_data = data5

In [ ]:
unscaled_data.head()

In [ ]:
unscaled_data.columns.values

In [ ]:
x2_unscaled=unscaled_data[['logCRIM','ZN','INDUS','RM','AGE','logDIS','RAD','PTRATIO','B','logLSTAT']]

In [ ]:
y2 = unscaled_data['MEDV']

In [ ]:
scaler2 = StandardScaler()

In [ ]:
scaler2.fit(x2_unscaled)

In [ ]:
x2_scaled = scaler2.transform(x2_unscaled)

In [ ]:
x2_scaled

### Create the train test split

In [ ]:
x2_train,x2_test, y2_train, y2_test = train_test_split(x2_scaled, y2, test_size=0.2,shuffle=True, random_state = 42)

In [ ]:
x2_train

In [ ]:
x2_train.shape, x2_test.shape,y2_train.shape, y2_test.shape

### creating the linear model

In [ ]:
model2 = LinearRegression()

In [ ]:
model2.fit(x2_train, y2_train)

In [ ]:
model2.score(x2_train, y2_train)

In [ ]:
r2_2 = model2.score(x2_train, y2_train)
n2 =  x2_train.shape[0]
p2 = x2_train.shape[1]
adj2_r2 = 1-((1-r2_2)*(n2-1)/(n2-p2-1))
adj2_r2

In [ ]:
test_score2 = model2.score(x2_test, y2_test)

In [ ]:
test_score2

In [ ]:
summary2 = pd.DataFrame()
features2 =['logCRIM','ZN','INDUS','RM','AGE','logDIS','RAD','PTRATIO','B','logLSTAT']
summary2['features'] = features2
summary2['Values']= model2.coef_
summary2


In [ ]:
model2.intercept_ 

Except intercept there is no difference in model

# Model 3

here we will not take logLSTAT instead we take only LSTAT

In [ ]:
data6 = data4[['logCRIM', 'ZN', 'INDUS', 'RM', 'AGE','logDIS', 'RAD', 'PTRATIO', 'B', 'LSTAT', 'MEDV']]

In [ ]:
data6.head()

### Standardisation of data

In [ ]:
unscaled_data3 = data6

In [ ]:
unscaled_data3.columns.values

In [ ]:
x3_unscaled = unscaled_data3[['logCRIM', 'ZN', 'INDUS', 'RM', 'AGE', 'logDIS', 'RAD', 'PTRATIO', 'B', 'LSTAT']]
y3 = unscaled_data3[ 'MEDV']

In [ ]:
scaler3 = StandardScaler()

In [ ]:
scaler3.fit(x3_unscaled)

In [ ]:
x3_scaled = scaler3.transform(x3_unscaled)

In [ ]:
x3_scaled

### Creating train test split

In [ ]:
x3_train,x3_test, y3_train, y3_test = train_test_split(x3_scaled, y3, test_size=0.2,shuffle=True, random_state = 42)

In [ ]:
x3_train.shape, x3_test.shape,y3_train.shape, y3_test.shape

### Creating the linear model

In [ ]:
model3 = LinearRegression()

In [ ]:
model3.fit(x3_train, y3_train)

In [ ]:
model3.score(x3_train, y3_train)

In [ ]:
r2_3 = model3.score(x3_train, y3_train)
n3 =  x3_train.shape[0]
p3 = x3_train.shape[1]
adj3_r2 = 1-((1-r2_3)*(n3-1)/(n3-p3-1))
adj3_r2

In [ ]:
test_score3 = model3.score(x3_test, y3_test)
test_score3

In [ ]:
summary3 = pd.DataFrame()
features3 =['logCRIM','ZN','INDUS','RM','AGE','logDIS','RAD','PTRATIO','B','LSTAT']
summary3['features'] = features3
summary3['Values']= model3.coef_
summary3

In [ ]:
model3.intercept_

# Model 4

Since logLSTAT is giving better results we will reinstate that and change some other parameter
lets drop missing values for LSTAT instead


### Treating missing values

In [ ]:
raw_data1 = pd.read_csv('../input/boston-housing-dataset/HousingData.csv')

In [ ]:
data7 = raw_data1

In [ ]:
data7.describe()

In [ ]:
data7['AGE']  = data7['AGE'].replace(np.NaN , data7['AGE'].mean())

In [ ]:
data7['ZN']  = data7['ZN'].replace(np.NaN , data7['ZN'].median())

In [ ]:
data7['INDUS']  = data7['INDUS'].replace(np.NaN , data7['INDUS'].median())

In [ ]:
data7['CRIM']  = data7['CRIM'].replace(np.NaN , data7['CRIM'].median())

In [ ]:
data7.describe()

In [ ]:
data8 = data7.drop(['CHAS'] , axis=1)

In [ ]:
data8.reset_index(drop=True)

In [ ]:
data8 = data8.dropna(axis=0)

In [ ]:
data8.describe()

In [ ]:
data8.reset_index(drop=True)

### Checking the OLS assumptions

In [ ]:
crimlog = np.log(data8['CRIM'])
dislog = np.log(data8['DIS'])
lstatlog = np.log(data8['LSTAT'])

In [ ]:
data8['logCRIM'],data8['logDIS'],data8['logLSTAT'] = crimlog,dislog, lstatlog

In [ ]:
data8.head()

In [ ]:
data8.columns.values

In [ ]:
data9 = data8[['logCRIM','ZN','INDUS','RM','AGE','logDIS','RAD','PTRATIO','B','logLSTAT','MEDV']]

In [ ]:
data9.head()

### Standardization of the data

In [ ]:
unscaled_data4 = data9
unscaled_data4.columns.values

In [ ]:
x4_unscaled = unscaled_data4[['logCRIM', 'ZN', 'INDUS', 'RM', 'AGE', 'logDIS', 'RAD', 'PTRATIO', 'B', 'logLSTAT']]
y4 = unscaled_data4[ 'MEDV']

In [ ]:
scaler4= StandardScaler()
scaler4.fit(x4_unscaled)
x4_scaled = scaler4.transform(x4_unscaled)

In [ ]:
x4_scaled

### Creating train test split

In [ ]:
x4_train,x4_test, y4_train, y4_test = train_test_split(x4_scaled, y4, test_size=0.2,shuffle=True, random_state = 42)

In [ ]:
x4_train.shape, x4_test.shape,y4_train.shape, y4_test.shape

### Creating the linear model

In [ ]:
model4 = LinearRegression()

In [ ]:
model4.fit(x4_train, y4_train)

In [ ]:
model4.score(x4_train, y4_train)

In [ ]:
r2_4 = model4.score(x4_train, y4_train)
n4 =  x4_train.shape[0]
p4 = x4_train.shape[1]
adj4_r2 = 1-((1-r2_4)*(n4-1)/(n4-p4-1))
adj4_r2

In [ ]:
test_score4 = model4.score(x4_test, y4_test)
test_score4

In [ ]:
summary4 = pd.DataFrame()
features4 =['logCRIM','ZN','INDUS','RM','AGE','logDIS','RAD','PTRATIO','B','logLSTAT']
summary4['features'] = features4
summary4['Values']= model4.coef_
summary4

In [ ]:
model4.intercept_

### Further analysis on feature selection¶

In [ ]:
from sklearn.feature_selection import f_regression

In [ ]:
f_regression(x4_train,y4_train)

In [ ]:
p_value = f_regression(x4_train,y4_train)[1].round(3)

In [ ]:
p_value

In [ ]:
f_value = f_regression(x4_train,y4_train)[0].round(3)
f_value

All existing parameters are relavent
